In [1]:
workspace()
type RNN
    Whh   # Initialize these with random values from normal distribution,
    Wxh   # scaled way down.
    Why
    bh
    by

    mWhh   # Initialize these with random values from normal distribution,
    mWxh   # scaled way down.
    mWhy
    mbh
    mby

    # TODO: remove h.
    h     # Not a variable, but still associated with the RNN.
    hidden_length
    
    function RNN(in_dim::Integer, hidden_dim::Integer, out_dim::Integer)
        # Initialize the Weights/Biases variables
        Whh = randn(hidden_dim, hidden_dim)*0.01
        Wxh = randn(hidden_dim, in_dim)*0.01
        Why = randn(out_dim, hidden_dim)*0.01
        bh = zeros(hidden_dim, 1)
        by = zeros(out_dim, 1)
        
        # Initialize the memory variables
        mWhh, mWxh, mWhy, mbh, mby = zeros(Whh),zeros(Wxh), zeros(Why), zeros(bh), zeros(by)
        
        h = zeros(bh)
        #hidden_length = randn(hidden_dim, 1),
        hidden_length = hidden_dim

        new(Whh, Wxh, Why, bh, by, mWhh, mWxh, mWhy, mbh, mby, h, hidden_length)
    end
end
r = RNN(1,2,1)
println(r.Wxh)
r.h

[0.0020326260805686542
 -0.011779828267081568]


2x1 Array{Float64,2}:
 0.0
 0.0

In [2]:
function clip_values(deltas_matrix, max_threshold::Float64, min_threshold::Float64)
    deltas_matrix = max(deltas_matrix, min_threshold)
    deltas_matrix = min(deltas_matrix, max_threshold)
    return deltas_matrix
end
clip_values([1.0 -11.0 6.0]', 5.0, -5.0)

3x1 Array{Float64,2}:
  1.0
 -5.0
  5.0

In [3]:
x = Dict{Int32,Float32}()
x[4] = 2.0
x
for i in enumerate(x)
    println(i)
end
for i in range(1,4)
    print(i) ; print(' ')
end

zeros([2 232 2; 2  2 1])
r.bh

(1,(4,2.0f0))
1 2 3 4 

2x1 Array{Float64,2}:
 0.0
 0.0

In [4]:
function forward_pass(r::RNN, in_h, x)
    local Wxh = r.Wxh
    local Whh = r.Whh
    local Why = r.Why
    local bh = r.bh
    local by = r.by

    # Calculate the new h.
    new_h = tanh(Wxh*x + Whh*in_h + bh)     # 3x1
    #println("new_h:$new_h")

    y = Why*new_h + by      # 2x1
    #println("y:$y")

    # exp(y) conveniently eliminates negative values from tanh.
    exp_y = exp(y)     # 2x1 (pointwise)
    
    y_norm = exp_y ./ sum(exp(y)) # 2x1 Element wise division 
    #println("y_norm: $y_norm")

    return y_norm, y, new_h
end
r1 = RNN(2,4,3)
r1.h = [0 0 0 0.]'
y1, _, r1.h = forward_pass(r1, r1.h, [1.0 0]')
y2, _, r1.h = forward_pass(r1, r1.h, [0.0 0.1]')
y3, _, r1.h = forward_pass(r1, r1.h, [1.0 0.]')
println("ys:\n$y1\n$y2\n$y3")
println("h:\n",r1.h)

ys:
[0.33330951534736897
 0.3333406406371714
 0.3333498440154596]
[0.33333748689492954
 0.33332865248686105
 0.33333386061820935]
[0.3333096100905018
 0.33334065387341205
 0.33334973603608614]
h:
[0.0031506157848285982
 0.0012353992743882975
 -0.009110897172611366
 -0.0004064513621215561]


In [5]:
ts=[1,2,3,4]
for i in length(ts):-1:1
    println(i)
end

4
3
2
1


In [6]:
# Hyperparamaters
hidden_size = 100
learning_rate = 1e-1
seq_length = 25

25

In [7]:
function forward_pass_backpropogate(r::RNN, xs, ts)
    """r - an RNN.
    x - The batch of input vectors: a tuple of vectors.
    t - The batch of "truth" vectors: a tuple of vectors of same count as xs."""

    #Forward Pass
    hs, ys, ps = Dict(), Dict(), Dict()
    hs[0] = r.h
    local Wxh = r.Wxh
    local Whh = r.Whh
    local Why = r.Why
    local bh = r.bh
    local by = r.by

    out_delta_max_threshold = 5.0   # Tweakable.
    out_delta_min_threshold = -5.0   # Tweakable.

    cost = 0
    
    for i in 1:length(ts)
        local h = hs[i-1]
        local x = xs[i]
        local t = ts[i]

        ps[i], ys[i], hs[i] = forward_pass(r, h, x)

        # Multiply by t-inverse, to select the value from log(ps[i]) which corresponds to the truth.
        cost += -t' * log(ps[i])  # 1x1 (scalar-ish) (log is pointwise).
        #println("cost:$cost")
    end

    # ----------  Now# go back down

    
    dWxh = zeros(r.Wxh)
    dWhh = zeros(r.Whh)
    dWhy = zeros(r.Why)
    dbh  = zeros(r.bh)
    dby  = zeros(r.by)
    
    #dhnext = zeros(r.h)
    ∂cost_∂hnext = zeros(r.h)
    for i in length(ts):-1:1

        # Copied from Karpathy. I don't know what these two lines mean.
        #dy = ps[i]
        #dy -= ts[i] # backprop into y
        #dWhy += dy * hs[i]'
        #dby += dy
        #dh = Why' * dy + dhnext # backprop into h
        #dhraw = (1 - hs[i] .* hs[i]) .* dh # backprop through tanh nonlinearity
        #dbh += dhraw
        #dWxh += dhraw * xs[i]'
        #dWhh += dhraw * hs[i-1]'
        #dhnext = Whh' * dhraw
        
        
        ∂cost_∂y = ps[i]
        ∂cost_∂y -= ts[i] # backprop into y
 
        ∂cost_∂Why = ∂cost_∂y * hs[i]'  # 2x3
        ∂cost_∂by = ∂cost_∂y            # 2x1
 
        ∂cost_∂h = Why' * ∂cost_∂y  +  ∂cost_∂hnext   # 3x1
        dhraw = (1 - hs[i] .* hs[i]) .* ∂cost_∂h    # "backprop through tanh nonlinearity"
 
        ∂cost_∂bh = dhraw  # 3x1    # noop
 
        ∂cost_∂Whh = dhraw * hs[i-1]'   # 3x3
        ∂cost_∂Wxh = dhraw * xs[i]'   # 3x2
 
        ∂cost_∂hnext = Whh' * dhraw  # 3x1
 
        # Update the final derivatives
 
        dWxh += ∂cost_∂Wxh
        dWhh += ∂cost_∂Whh
        dWhy += ∂cost_∂Why
        dbh  += ∂cost_∂bh
        dby  += ∂cost_∂by


        #println("∂cost_∂Wxh:$∂cost_∂Wxh")
    end

    # Clip deltas to mitigate exploding gradients.
    dWxh = clip_values(dWxh, out_delta_max_threshold, out_delta_min_threshold) 
    dWhh = clip_values(dWhh, out_delta_max_threshold, out_delta_min_threshold)
    dWhy = clip_values(dWhy, out_delta_max_threshold, out_delta_min_threshold)
    dbh = clip_values(dbh, out_delta_max_threshold, out_delta_min_threshold)
    dby = clip_values(dby, out_delta_max_threshold, out_delta_min_threshold)
     
    # Update the hidden state from this run.
    r.h = hs[length(ts)]

    return cost, dWxh, dWhh, dWhy, dbh, dby 

end

r = RNN(2,3,2)
r.Wxh = [.5 .2 ; 0.1 0.1 ; 0.2 0.2]
r.Whh = [.1 .1 .1 ; .2 .2 .2 ; 0.3 0.3 .3]
r.Why = [.4 .5 .6; .7 .8 .9 ]
r.h = [0.4 .2 0.8]'
println(forward_pass_backpropogate(r, ([1 0]', [0 1]'), ([0 1]', [1 0]')))
println("\n",r.h)

(
[1.3988750128749587],

[-0.02348709404610685 0.15845008784877856
 -0.029956754210863596 0.15314729364197688
 -0.02401725804279269 0.12098114381203691],

[0.08011354615577733 0.052773611291928826 0.06853660930090805
 0.07453013601361681 0.049556316201140885 0.06043836265216451
 0.058735290825127406 0.03907731268820138 0.04746229284518379],

[0.02188565806931217 -0.08201489754933375 -0.12198683957866582
 -0.021885658069312197 0.08201489754933372 0.1219868395786658],

[0.1349629938026717
 0.12319053943111329
 0.09696388576924422],

[-0.20382526255910166
 0.2038252625591016])

[0.3344883118838543
 0.37630407100318514
 0.5673596773818216]


In [8]:
xs = ([0. 1.]', [0. 1.]')
xs[1]

2x1 Array{Float64,2}:
 0.0
 1.0

In [9]:
max([1.0 -11.0 5.0]', -5)

3x1 Array{Float64,2}:
  1.0
 -5.0
  5.0

In [10]:
test_r = RNN(5,20,6)
print("big")

big

In [11]:
forward_pass_backpropogate(test_r, ([1 10 1 3 2]',), ([2 1. 8 0.3 2 0.1]',))
print("Testing the print statements.")

Testing the print statements.

Training!
---------
Now let's do the training!

Concept:

Calculate the gradient of the Cost with respect to each of the weights, i.e. the partial derivates $\partial$Cost / $\partial$Weight for each weight.

This gradient represents how much the Cost would change if we update the weights a tiny bit. So we subtract the gradient from the weight to decrease the Cost!

Note: We don't even care what $y$ is, we only use it to calculate the gradient for a given set of weights. So that information can stay in the backpropogate function.

In [12]:
function update(r, xs, ts)
    """r - RNN, xs - a batch of inputs, ts - a batch of truths (as one-hot nx1 matrices)"""
    loss, ∂cost_∂Wxh, ∂cost_∂Whh, ∂cost_∂Why, ∂cost_∂bh, ∂cost_∂by = forward_pass_backpropogate(r,xs,ts)
    
    # adagrad update (Gradient Descent)
    r.mWxh += ∂cost_∂Wxh .^ 2
    r.mWhh += ∂cost_∂Whh .^ 2
    r.mWhy += ∂cost_∂Why .^ 2
    r.mbh += ∂cost_∂bh .^ 2
    r.mby += ∂cost_∂by .^ 2
    
    r.Wxh -= learning_rate * ∂cost_∂Wxh  ./ sqrt(r.mWxh + 1e-8)
    r.Whh -= learning_rate * ∂cost_∂Whh  ./ sqrt(r.mWhh + 1e-8)
    r.Why -= learning_rate * ∂cost_∂Why  ./ sqrt(r.mWhy + 1e-8)
    r.bh -= learning_rate * ∂cost_∂bh    ./ sqrt(r.mbh + 1e-8)
    r.by -= learning_rate * ∂cost_∂by    ./ sqrt(r.mby + 1e-8)
    
    return loss
end

r = RNN(2,3,2)
r.Wxh = [.5 .2 ; 0.1 0.1 ; 0.2 0.2]
r.Whh = [.1 .1 .1 ; .2 .2 .2 ; 0.3 0.3 .3]
r.Why = [.4 .5 .6; .7 .8 .9 ]
r.h = [0.4 .2 0.8]'
loss = update(r, ([1 0]', [0 1]', [0 1]', [1 0]'), ([0 1]', [1 0]', [0 1]', [1 0]'))
println(loss, r.h)

update(r, ([1 0]', [0 1]', [0 1]', [1 0]'), ([0 1]', [1 0]', [0 1]', [1 0]'))
update(r, ([1 0]', [0 1]', [0 1]', [1 0]'), ([0 1]', [1 0]', [0 1]', [1 0]'))
loss = update(r, ([1 0]', [0 1]', [0 1]', [1 0]'), ([0 1]', [1 0]', [0 1]', [1 0]'))

println(loss, r.h)

println(r.Wxh, r.Whh, r.Why, r.bh, r.by)

[2.8513927492820152][0.5499489269489144
 0.32445312710513785
 0.5042017549258068]
[2.7732563220837685][0.2733504302372906
 -0.1218558122595025
 -0.03313777320467589]
[0.3673099058722714 0.10641030522686429
 -0.026636732258434413 0.006942351574688553
 0.06798023469772287 0.11654742554190307][0.014194231453489218 0.02240190796108634 0.015019502381031684
 0.11765274426740126 0.12166334880633656 0.11498598332296982
 0.22201671040103732 0.22457719522088102 0.2173526964776151][0.4753469640869792 0.5827236545826828 0.6514414354375764
 0.6246530359130209 0.7172763454173171 0.8485585645624236][-0.11227657394104021
 -0.11147721367622182
 -0.11101694366027448][0.021547450149868995
 -0.021547450149868946]


##The Data

###Shakespeare's Comedies


In [36]:
shakespeare_text = ""
open("all_comedies_cat.txt") do shakespeare_full_text_file
    global shakespeare_text
    shakespeare_text = readall(shakespeare_full_text_file)
end
println("Length: ", length(shakespeare_text))
println("\n --------- Some Text: -------- \n")
println(shakespeare_text[1:150])
println("\n --------- .... -------- \n")
print(shakespeare_text[end-1000:end])

Length: 2108622

 --------- Some Text: -------- 

	ALL'S WELL THAT ENDS WELL


	DRAMATIS PERSONAE


KING OF FRANCE	(KING:)

DUKE OF FLORENCE	(DUKE:)

BERTRAM	Count of Rousillon.

LAFEU	an old lord.

P

 --------- .... -------- 

y mate, that's never to be found again,
	Lament till I am lost.

LEONTES	O, peace, Paulina!
	Thou shouldst a husband take by my consent,
	As I by thine a wife: this is a match,
	And made between's by vows. Thou hast found mine;
	But how, is to be question'd; for I saw her,
	As I thought, dead, and have in vain said many
	A prayer upon her grave. I'll not seek far--
	For him, I partly know his mind--to find thee
	An honourable husband. Come, Camillo,
	And take her by the hand, whose worth and honesty
	Is richly noted and here justified
	By us, a pair of kings. Let's from this place.
	What! look upon my brother: both your pardons,
	That e'er I put between your holy looks
	My ill suspicion. This is your son-in-law,
	And son unto the king, who, heavens directing,
	

In [14]:
alphabet = sort(unique(shakespeare_text))
size_alpha = length(alphabet)
println(size_alpha)

reverse_alphabet = [ch => i for (i,ch) in enumerate(alphabet)]

69


Dict{Any,Int64} with 69 entries:
  'D'  => 18
  '|'  => 69
  'Y'  => 39
  '\'' => 6
  '.'  => 11
  'U'  => 35
  'B'  => 16
  ':'  => 12
  ';'  => 13
  'J'  => 24
  'Z'  => 40
  'o'  => 57
  'N'  => 28
  'p'  => 58
  'F'  => 20
  'j'  => 52
  '!'  => 4
  'y'  => 67
  'E'  => 19
  'r'  => 60
  'm'  => 55
  'S'  => 33
  'A'  => 15
  ','  => 9
  'T'  => 34
  ⋮     => ⋮

In [15]:
function make_one_hot(length, index)
    v = zeros(Float64, length, 1)
    v[index] = 1.0
    return v
end
x = make_one_hot(length(alphabet), reverse_alphabet['.'])
assert(1 == x[reverse_alphabet['.']])
assert(0 == x[reverse_alphabet['a']])

In [16]:
# Because Julia doesn't seem to have a function to sample from a set of probabilities?

function rand_uniform(a, b)
    a + rand()*(b - a)
end
function SampleFrom(probabilities)

    # Sum to create CDF:
    cdf = Array(Float64, 0)
    sum = 0.0
    for p in probabilities
        push!(cdf, sum + p)
        sum = cdf[end];
    end
        
    # Choose from CDF:
    cdf_value = rand_uniform(0.0,cdf[end])
    index = searchsortedfirst(cdf, cdf_value);

    return index;

end
println(SampleFrom([1 1 1 1]))
println(SampleFrom([1 1 10 10]))
println(SampleFrom([0.1 0.2 0.7 0.8]))
#println(hist([SampleFrom([1 1 10 10]) for i in 0:1000]))


2
4
3


In [17]:
function hallucinate(r, seed_idx, num_chars)
    hallucination = ""
    prev_ids = [seed_idx]
    # clear hidden state (not the weights)
    #r.h = zeros(Float64, r.hidden_length, 1) # or maybe not...
    # Karpathy doesn't modify h.
    for x in range(1,num_chars)
        x_vec = make_one_hot(length(alphabet), prev_ids[end])
        y_norm,y,r.h = forward_pass(r, r.h, x_vec)

        # Now sample from y!
        letter_idx = SampleFrom(y_norm')
        #letter_idx = indmax(y)
        char = alphabet[letter_idx]
        
        append!(prev_ids,[letter_idx])
        hallucination = "$hallucination$char"
    end
    return hallucination
end
r = RNN(length(alphabet), 500, length(alphabet))
hallucinate(r, rand(1:length(alphabet)), 100)

"E)u.ADDgYn|i;X\nxVWzYmn,CpZ;;KE-y( kT|JhMxGlSjd;!\nV'da&YKJ;OwC]tG\t|aXhUkVkj:f?f KfMAENmbqZJibSZ(LofFQ"

In [18]:
training_rnn = RNN(length(alphabet), hidden_size, length(alphabet))
chars_trained = 1
println("======== Chars Trained:", chars_trained-1, "/", length(shakespeare_text), "===========")
println(hallucinate(training_rnn, 1, 100))

======== Chars Trained:0/2108622===========
RXZcEFcfZqKbfD)Q-')Mif)UC!ynFNctLjsl.dWDUdIbs!NWjZnQB(aCFI;cqC.Mf,tb?WYVQkDTN|p&
(N-MtxNw )Z&)i;(K?A


In [19]:
#for seq_length in [20,100,200,400]#,20,20,20]
i = 0
smooth_loss = -log(1.0/length(alphabet))*seq_length # loss at iteration 0
println("======== Iteration: $i, Chars Trained:", chars_trained-1, "/", length(shakespeare_text), " Cost: $smooth_loss ===========")
while true
    if chars_trained+seq_length+1 >= length(shakespeare_text)
        println("========= DONE!! ===========")
        break
    end
    x_vecs = [make_one_hot(length(alphabet), reverse_alphabet[x]) for x in shakespeare_text[chars_trained:chars_trained+seq_length]] 
    truth_vecs = [make_one_hot(length(alphabet), reverse_alphabet[y_]) for y_ in shakespeare_text[chars_trained+1:chars_trained+seq_length+1]]

    if i % 100 == 0
        println("======== Iteration: $i, Chars Trained:", chars_trained-1, "/", length(shakespeare_text), " Cost: $smooth_loss ===========")
        seed_idx = reverse_alphabet[shakespeare_text[chars_trained]]
        println(hallucinate(training_rnn, seed_idx, 100))
    end

    (loss,) = update(training_rnn, x_vecs, truth_vecs)
    smooth_loss = smooth_loss * 0.999 + loss * 0.001

    chars_trained += seq_length
    i += 1

end

======== Iteration: 0, Chars Trained:0/2108622 Cost: 105.85266261493149 ===========
======== Iteration: 0, Chars Trained:0/2108622 Cost: 105.85266261493149 ===========
tDmV[Ft;)BTU k?;i[A;qTgs
-	QYaNzndhQzdQl'.	AIVx(kgXAEQ	BJDsCSNUoleBPcZ
''.O:.XvX)orovVzXKi]BgmkVuops
======== Iteration: 100, Chars Trained:2500/2108622 Cost: 107.16171946695124 ===========
aiLt& sh|diuAeibryv]tE-RW T
NEVnI )V!  wvL.[a
rBTo atooldKNdi(? ,rtmUeBgtts
OBWYsS y ,

e ihE

 tn,Y
======== Iteration: 200, Chars Trained:5000/2108622 Cost: 105.50344564310039 ===========
W s
	  n  
TIrhceg)Ue
	m-e  oAa. cbhlrgr eho tto]dn ttde  tEI r
:cug,neeesnUcihf r ;oCE
rdUUoAnoiLea
======== Iteration: 300, Chars Trained:7500/2108622 Cost: 103.51962100847167 ===========
isiF tlohnn ioiwecF Myu?  rweTy itltnefdfsdeorer,  d &aeaetMeoregToligde smrt llhyot, and t ugeoaosd
======== Iteration: 400, Chars Trained:10000/2108622 Cost: 101.52140245180223 ===========
hwsiinhhn ghw hme hl!

LuA
HLEUNNAopp
RWP n t lacAe p uthnton 	calhnlTg

LoadError: interrupt
while loading In[19], in expression starting on line 5

In [22]:
println(hallucinate(training_rnn, rand(1:length(alphabet)), 5000))

fhat and ander broos by I hend dot sass the dith; the kam,
	Thishe
	And;
	Mar eave oorsf ther na. Hils ir?

DRSAKEUS IUh E

OUNUEHEwOSALINDEN	That sires my, to prootstharen Afeers save wind the sep,
	Dot liads Sald,
	Work weat bo wistires Anour pristed, will trens with acpering!
	H t, mithid.

SALIN]
	'thed she pnrt, fathe orto dahis, eave ond go bedold mearsaid you, afure lotrs hourd moury nike the muse,
	He hamed,
	Ar, to har the'd
	Th my sarss tor!

	To prowied: qucheny soxes boniast, till,
	Woml waw fiallf.

	[Ericedou.
	hyam merfe, we we sold
	Bir willt canou bet boL my reblnt Somein, Dy Sippe, amit
	Thimg,
	Thourr verincu, ast raperasurfes.
	I mave, theaw:




TOSCKE SHSARBIN E ly I shouris the pees pracithel bind'nge this so no cof bacvesile To thoung St I spoun hcom'd
	Her,
	Youand hith
	Of, ay disctilsd; os'se murs.

SALIA	Iou moay broud weatheco's
	Ffostantinnd ard ves wourren, ammus,;
	End cheng? tobe fi the
	This ard as af instorn
	Thate dime, Modeds be; numbrove fou's warv

In [ ]:
#using PyPlot
#plot(total_costs, linewidth=2.0, linestyle="-")


In [ ]:
#using Gadfly
#u#sing DataFrames
#
#samples = [SampleFrom([1 1 10 10]) for i in 0:1000]
#_, count = hist(samples)
#class = sort(unique(samples))
#value_counts = DataFrame(class=class, count=count)
#
#plot(value_counts, x="class", y="count", Geom.bar)

In [ ]:
#?Gadfly

In [ ]:
function count_extreme_values(mat)
    num_ones = 0
    num_neg_ones = 0
    count = 0
    for x in mat
        if x <= -1
            num_neg_ones += 1
        elseif x >= 1
            num_ones = 1
        end
        count += 1
    end
    num_ones, num_neg_ones, count
end

In [ ]:
count_extreme_values(r.h)

In [ ]:
r.h

In [ ]:
print([i for i in range(0,2,5)])

**1/15/16**, When it first started learning!:

```
======== Chars Trained:1/2108622===========
gt!(,Xsxcn'aaivTLlAnTwerS-Ndv-i	A[q.nT'tTniRg[z?l?v[N(TW&&!CH-XYP.K]wGZ;g.S(r(X?gJ|I[&Ne-X(([cHTggZ	
======== Chars Trained:2/2108622===========
PJwHcFP(]PA,PIPEto
]P(TsN,(iZg	-b[!	z,][iMJga],wjIRGlMS..-((Tbv][jWs]ATmWvO
]w.,tiPKVdE;[tjvXia.(j
P
======== Chars Trained:3/2108622===========
C[-X jrWDBAzj[P:PXZgN?-['V,]fxk[xP;Qen-[dmMv'GmE-i!PgpR(rMP|ifLe-EjgclgTwx[T'PRdwB'!CtR]kPBPgKG
s?
D
======== Chars Trained:4/2108622===========
[	Dg-Pm!?yC(Jg?WG)JTZ-Ovngn.	iWP(	fFp	agCMX	bwjETc,Sw&XKiSZAvdxICR[g	.Pjy'W[!U&&.)Y'PFXz(fP	HPZry
(b
======== Chars Trained:6/2108622===========
W  	F  F(   	  g 	g(   		g  iF   d     F     F  g    	gB  F     F   b  b 	   F 	F  b      gw	 		    
======== Chars Trained:8/2108622===========
																																																																																																				
======== Chars Trained:11/2108622===========
||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
======== Chars Trained:16/2108622===========
||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||

```

**After some digging**, it turns out of course that there are NaNs again. I thought maybe the reason it's selecting the '|', or the last character, is that the y vector is all even. (Note, I'm not sampling, but choosing the max). And, in some sense it is:

```
x = make_one_hot(length(alphabet), reverse_alphabet['a'])
t = make_one_hot(length(alphabet), reverse_alphabet['b'])
forward_pass_backpropogate(r, x, t)

Out[152]: (
69x1 Array{Float64,2}:
 NaN
 NaN
   ⋮
 NaN
 NaN,

1400x69 Array{Float64,2}:
 NaN …  NaN
   ⋮ ⋱    ⋮
 NaN …  NaN,

1400x1400 Array{Float64,2}:
 NaN …  NaN
   ⋮ ⋱    ⋮
 NaN …  NaN,

69x1400 Array{Float64,2}:
 NaN …  NaN
   ⋮ ⋱    ⋮
 NaN …  NaN,
 
1400x1 Array{Float64,2}:
 NaN
 NaN
   ⋮
 NaN
 NaN)

```

##UNIT TEST

From Karpathy's python code:
```
# hyperparameters
hidden_size = 3 # size of hidden layer of neurons
vocab_size = 2

# model parameters
Wxh = np.array([[.5, .2], [0.1, 0.1], [0.2, 0.2]])
Whh = np.array([[.1, .1, .1], [.2, .2, .2], [.3, .3, .3]])
Why = np.array([[.4, .5, .6], [.7, .8, .9]])
bh = np.zeros((hidden_size, 1)) # hidden bias
by = np.zeros((vocab_size, 1)) # output bias

h = [[0.4], [.2], [0.8]]

for x in lossFun([0,1], [1,0], h):
    print x, "\n"
```

Out:
```
1.39887501287 

[[-0.02348709  0.15845009]
 [-0.02995675  0.15314729]
 [-0.02401726  0.12098114]] 

[[ 0.08011355  0.05277361  0.06853661]
 [ 0.07453014  0.04955632  0.06043836]
 [ 0.05873529  0.03907731  0.04746229]] 

[[ 0.02188566 -0.0820149  -0.12198684]
 [-0.02188566  0.0820149   0.12198684]] 

[[ 0.13496299]
 [ 0.12319054]
 [ 0.09696389]] 

[[-0.20382526]
 [ 0.20382526]] 

[[ 0.33448831]
 [ 0.37630407]
 [ 0.56735968]] 
```

In [ ]:
r = RNN(2,3,2)
r.Wxh = [.5 .2 ; 0.1 0.1 ; 0.2 0.2]
r.Whh = [.1 .1 .1 ; .2 .2 .2 ; 0.3 0.3 .3]
r.Why = [.4 .5 .6; .7 .8 .9 ]
r.h = [0.4 .2 0.8]'
outs = forward_pass_backpropogate(r, ([1 0]', [0 1]'), ([0 1]', [1 0]'))
truth = (
[1.3988750128749587]',

[-0.02348709404610685 0.15845008784877856
 -0.029956754210863596 0.15314729364197688
 -0.02401725804279269 0.12098114381203691],

[0.08011354615577733 0.052773611291928826 0.06853660930090805
 0.07453013601361681 0.049556316201140885 0.06043836265216451
 0.058735290825127406 0.03907731268820138 0.04746229284518379],

[0.02188565806931217 -0.08201489754933375 -0.12198683957866582
 -0.021885658069312197 0.08201489754933372 0.1219868395786658],

[0.1349629938026717
 0.12319053943111329
 0.09696388576924422]'',

[-0.20382526255910166
 0.2038252625591016]'')

for i in range(1,length(outs))
    if outs[i] != truth[i]
        println("Incorrect! Outs:\n", outs[i], "\nvs\n", truth[i])
    end
end

true_r_h = [0.3344883118838543 0.37630407100318514 0.5673596773818216]'
if r.h != true_r_h
    println("Incorrect! r.h:\n", r.h, "\nvs\n", true_r_h)
end